In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.2 MB/s eta 0:00:00


# Primera Tanda

In [ ]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker('es_ES')  # Localización peruana

# Parámetros base
NUM_REGISTROS = 150
MESAS_IDS = list(range(1, 11))        # 10 mesas
TRABAJADORES_IDS = list(range(1, 9))  # 8 trabajadores
METODOS_PAGO_IDS = [1, 2, 3]          # 3 métodos de pago

In [ ]:
# ---------- Generar PEDIDOS ----------
pedidos = []
for i in range(11, NUM_REGISTROS + 11):
    fecha = fake.date_time_between(start_date='-30d', end_date='now')
    pedidos.append({
        'id': i,
        'total': 0.00,  # se calculará en factura
        'entregado': 1,
        'fecha': fecha,
        'mesa_id': random.choice(MESAS_IDS),
        'trabajador_id': random.choice(TRABAJADORES_IDS)
    })

df_pedido = pd.DataFrame(pedidos)

In [ ]:
df_pedido

,id,total,entregado,fecha,mesa_id,trabajador_id
0,11,21.69,1,2025-06-17 22:53:29.753730,9,7
1,12,77.17,1,2025-06-07 15:43:28.916858,8,6
2,13,33.58,1,2025-06-22 07:18:14.933001,6,8
3,14,77.63,1,2025-06-11 10:25:02.026271,4,2
4,15,75.06,1,2025-07-03 14:56:57.229585,6,2
...,...,...,...,...,...,...
145,156,56.97,1,2025-06-12 05:21:22.234179,2,4
146,157,73.84,1,2025-06-29 21:56:31.563134,6,8
147,158,43.15,1,2025-07-03 05:22:32.856426,1,2
148,159,53.61,1,2025-06-12 10:48:42.697993,6,3


In [ ]:
# ---------- Generar FACTURAS ----------
facturas = []
for pedido in pedidos:
    subtotal = round(random.uniform(20.0, 80.0), 2)
    descuento = round(random.choice([0.0, 2.5, 5.0]), 2)
    igv = round(0.18 * (subtotal - descuento), 2)
    monto_final = round((subtotal - descuento) + igv, 2)

    facturas.append({
        'id': pedido['id'],
        'monto_final': monto_final,
        'descuento': descuento,
        'igv': igv,
        'pagado': 1,
        'pedido_id': pedido['id']
    })

df_factura = pd.DataFrame(facturas)

# Actualizar el total del pedido con el mismo monto
df_pedido['total'] = df_factura['monto_final']

df_factura

,id,monto_final,descuento,igv,pagado,pedido_id
0,11,21.69,2.5,3.31,1,11
1,12,77.17,2.5,11.77,1,12
2,13,33.58,2.5,5.12,1,13
3,14,77.63,0.0,11.84,1,14
4,15,75.06,0.0,11.45,1,15
...,...,...,...,...,...,...
145,156,56.97,2.5,8.69,1,156
146,157,73.84,0.0,11.26,1,157
147,158,43.15,5.0,6.58,1,158
148,159,53.61,2.5,8.18,1,159


In [ ]:
# ---------- Generar PAGOS ----------
pagos = []
for factura in facturas:
    pagos.append({
        'id': factura['id'],
        'numero_transaccion': fake.bothify(text='TRX-#####'),
        'fecha': fake.date_time_between(start_date='-30d', end_date='now'),
        'factura_id': factura['id'],
        'metodo_pago_id': random.choice(METODOS_PAGO_IDS)
    })

df_pago = pd.DataFrame(pagos)
df_pago

,id,numero_transaccion,fecha,factura_id,metodo_pago_id
0,11,TRX-05393,2025-07-04 04:51:28.589370,11,2
1,12,TRX-01907,2025-06-16 23:07:00.312304,12,3
2,13,TRX-07064,2025-06-24 20:50:12.445970,13,1
3,14,TRX-45422,2025-06-26 20:44:04.584156,14,1
4,15,TRX-76498,2025-06-09 12:35:18.449776,15,1
...,...,...,...,...,...
145,156,TRX-64506,2025-07-02 14:10:39.623403,156,1
146,157,TRX-88411,2025-06-20 22:04:07.585248,157,1
147,158,TRX-30249,2025-07-03 12:42:53.620005,158,2
148,159,TRX-83188,2025-07-05 15:24:49.510174,159,2


In [ ]:
# ---------- Guardar CSVs ----------
df_pedido.to_csv('pedidos.csv', index=False)
df_factura.to_csv('facturas.csv', index=False)
df_pago.to_csv('pagos.csv', index=False)

print("¡Datos generados y guardados como CSV!")

¡Datos generados y guardados como CSV!


# Segunda Tanda

In [ ]:
random.seed(42)

# ---------------------------
# Parámetros
# ---------------------------

NUM_PEDIDOS = 150
NUM_TRABAJADORES = 8

PRODUCTOS = {
    1: (10.0, 1),   # Caldo de Gallina
    2: (8.0, 1),    # Causa Rellena
    3: (17.0, 2),   # Lomo Saltado
    4: (15.0, 2),   # Aji de Gallina
    5: (14.0, 2),   # Tallarines Verdes
    6: (10.0, 3),   # Suspiro a la Limena
    7: (8.0, 3),    # Mazamorra Morada
    8: (12.0, 4),   # Jarra de Chicha Morada
    9: (10.0, 4),   # Jarra de Limonada
    10: (10.0, 1),  # Tequeños de Queso
    11: (12.0, 1),  # Papa Rellena
    12: (22.5, 2),  # Seco de Cordero
    13: (18.0, 2),  # Chaufa de Pollo
    14: (9.5, 3),   # Torta Helada
    15: (11.0, 3),  # Helado de Lucuma
    16: (6.0, 4),   # Inka Cola 500ml
    17: (5.5, 4)    # Café pasado
}

OFERTAS = {
    3: 38.25,   # Combo Familiar Andino
    4: 32.00,   # Trío Criollo
    5: 23.80,   # Dulce Refrescante
    6: 29.75,   # Sabor de la Sierra
    7: 25.50    # Merienda para Dos
}

TURNOS_POSIBLES = [
    ("08:00", "16:00"),
    ("14:00", "22:00"),
    ("18:00", "02:00")  # cuidado con la fecha
]

In [ ]:
# ---------------------------
# 1. pedido_producto
# ---------------------------
pedido_producto = []
pedido_id = 1
detalle_id = 15

for pedido_id in range(11, NUM_PEDIDOS + 11):
    num_productos = random.choices([1, 2, 3], weights=[60, 30, 10])[0]
    productos_seleccionados = random.sample(list(PRODUCTOS.keys()), k=num_productos)

    for producto_id in productos_seleccionados:
        cantidad = random.randint(1, 2)
        precio = PRODUCTOS[producto_id][0]
        categoria = PRODUCTOS[producto_id][1]

        # Nota (según tipo)
        nota = None
        if random.random() < 0.12:  # ~12% chance de tener nota
            if categoria == 4:
                nota = random.choice(["con hielo", "sin hielo"])
            elif categoria == 2:
                nota = random.choice(["poca sal", "templada"])

        pedido_producto.append({
            "id": detalle_id,
            "cantidad": cantidad,
            "precio": precio,
            "producto_id": producto_id,
            "pedido_id": pedido_id,
            "notas": nota
        })
        detalle_id += 1

df_pedido_producto = pd.DataFrame(pedido_producto)
df_pedido_producto

,id,cantidad,precio,producto_id,pedido_id,notas
0,15,1,5.5,17,11,None
1,16,2,9.5,14,12,None
2,17,2,15.0,4,13,None
3,18,2,12.0,11,14,None
4,19,1,10.0,10,15,None
...,...,...,...,...,...,...
222,237,2,12.0,11,157,None
223,238,2,6.0,16,158,None
224,239,1,17.0,3,159,None
225,240,2,12.0,8,159,None


In [ ]:
df_pedido_producto.to_csv("pedido_producto.csv", index=False)
print("✅ pedido_producto.csv generado")

✅ pedido_producto.csv generado


In [ ]:
# ---------------------------
# 2. pedido_oferta
# ---------------------------
pedido_oferta = []
detalle_id = 4
oferta_pedidos = random.sample(range(11, NUM_PEDIDOS + 11), k=35)

for pedido_id in oferta_pedidos:
    oferta_id = random.choice(list(OFERTAS.keys()))
    precio = OFERTAS[oferta_id]
    cantidad = 1

    pedido_oferta.append({
        "id": detalle_id,
        "cantidad": cantidad,
        "precio": precio,
        "oferta_id": oferta_id,
        "pedido_id": pedido_id,
        "notas": None
    })
    detalle_id += 1

df_pedido_oferta = pd.DataFrame(pedido_oferta)
df_pedido_oferta

,id,cantidad,precio,oferta_id,pedido_id,notas
0,4,1,38.25,3,153,None
1,5,1,38.25,3,71,None
2,6,1,38.25,3,60,None
3,7,1,25.50,7,41,None
4,8,1,32.00,4,146,None
5,9,1,32.00,4,35,None
6,10,1,38.25,3,103,None
7,11,1,29.75,6,149,None
8,12,1,25.50,7,140,None
9,13,1,38.25,3,67,None


In [ ]:
df_pedido_oferta.to_csv("pedido_oferta.csv", index=False)
print("✅ pedido_oferta.csv generado")

✅ pedido_oferta.csv generado


In [ ]:
# ---------------------------
# 3. turno
# ---------------------------
turnos = []
turno_id = 3
fecha_base = datetime(2025, 7, 1)

for trabajador_id in range(1, NUM_TRABAJADORES + 1):
    fechas_utilizadas = set()
    for _ in range(5):
        # Evitar días repetidos
        while True:
            dia = random.randint(0, 29)
            if dia not in fechas_utilizadas:
                fechas_utilizadas.add(dia)
                break

        fecha_turno = fecha_base + timedelta(days=dia)
        hora_inicio, hora_fin = random.choice(TURNOS_POSIBLES)

        inicio_dt = datetime.combine(fecha_turno.date(), datetime.strptime(hora_inicio, "%H:%M").time())
        if hora_fin == "02:00":
            fin_dt = datetime.combine(fecha_turno + timedelta(days=1), datetime.strptime(hora_fin, "%H:%M").time())
        else:
            fin_dt = datetime.combine(fecha_turno.date(), datetime.strptime(hora_fin, "%H:%M").time())

        turnos.append({
            "id_turno": turno_id,
            "id_trabajador": trabajador_id,
            "inicio_turno": inicio_dt.strftime("%Y-%m-%d %H:%M:%S"),
            "fin_turno": fin_dt.strftime("%Y-%m-%d %H:%M:%S"),
        })
        turno_id += 1

df_turno = pd.DataFrame(turnos)
df_turno

,id_turno,id_trabajador,inicio_turno,fin_turno
0,3,1,2025-07-15 14:00:00,2025-07-15 22:00:00
1,4,1,2025-07-08 08:00:00,2025-07-08 16:00:00
2,5,1,2025-07-18 08:00:00,2025-07-18 16:00:00
3,6,1,2025-07-23 18:00:00,2025-07-24 02:00:00
4,7,1,2025-07-14 08:00:00,2025-07-14 16:00:00
5,8,2,2025-07-05 14:00:00,2025-07-05 22:00:00
6,9,2,2025-07-10 08:00:00,2025-07-10 16:00:00
7,10,2,2025-07-21 08:00:00,2025-07-21 16:00:00
8,11,2,2025-07-01 14:00:00,2025-07-01 22:00:00
9,12,2,2025-07-22 14:00:00,2025-07-22 22:00:00


In [ ]:
df_turno.to_csv("turno.csv", index=False)
print("✅ turno.csv generado")

✅ turno.csv generado
